In [1]:
import openai
import os
import fnmatch
from time import time, sleep
import textwrap
import re

openai.api_key = os.getenv("PRIVATE_OPENAI_KEY")


def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

def save_file(content, filepath):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

def read_files_recursively(dir_path):
    file_data = []
    for root, _, filenames in os.walk(dir_path):
        for filename in fnmatch.filter(filenames, '*.ts'):
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, dir_path)
            file_contents = open_file(file_path)
            file_data.append((relative_path, file_contents))
    return file_data


In [2]:
file_contents = read_files_recursively('src/routes/api/')

In [3]:
sveltekit_docs = open_file('sveltekit_docs.md')
prisma_schema = open_file('prisma/schema.prisma')

In [8]:
for path, content in file_contents:
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a system that can generate documentation for a SvelteKit project. This is the most recent sveltekit documentation on server api routes: " + sveltekit_docs + "\n" + "This is the most recent prisma schema for the app: " + prisma_schema},
            {"role": "user", "content": "Create api documentation in markdown with tables for request and response paramters with field, type, required (only for request) and description columns for the following api route: /api/" + str(path) + ". It has the following functions in TypeScript: " + str(content) },
        ],
    )
    completion.usage.total_tokens
    docs = open_file('generated_docs.md')
    save_file(docs + "\n" + completion.choices[0].message.content, 'generated_docs.md')

In [13]:
completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a system that should generate ERDs based on prisma schemas."},
        {"role": "user", "content": "Generate an Entity Relation Diagram in markdown format, also draw relationships between entities, for the following prisma schema: " + prisma_schema},
    ],
)
completion.usage.total_tokens


2402

In [12]:
erd = completion.choices[0].message.content
save_file(erd, 'generated_erd.md')